In [ ]:
import pandas as pd
from collections import Counter
from tqdm import tqdm
from nltk.tokenize import TweetTokenizer
import nltk
nltk.download('stopwords')
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
import re
import gensim
import gensim.corpora as corpora
from pprint import pprint
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /mnt/vocwork1/ddd_v1_w_na
[nltk_data]     X_1038985/asn661952_1/asn661953_1/work/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Data Cleaning

In [2]:
twitter_2020_02 = pd.read_csv("data/2020-02.csv", lineterminator="\n")

In [3]:
print(twitter_2020_02.shape)
file = twitter_2020_02.copy()

(347454, 18)


### Data Selection

In [4]:
# file.head(1)

In [5]:
file["full_text"] = file["full_text"].astype(str).replace("nan", "")
file["full_text.1"] = file["full_text.1"].astype(str).replace("nan", "")
file["full_text.2"] = file["full_text.2"].astype(str).replace("nan", "")
file.loc[file['full_text']+file["full_text.1"]+file["full_text.2"] == '', 'full_text'] = file['text']
file.insert(1, "combined_text", file["full_text"] + file["full_text.2"] + " " + file["full_text.1"])
file["rt_status"] = [0] * len(file)
file.loc[file["text"].str.startswith("RT"), "rt_status"] = 1

In [6]:
#drop duplicate rows, e.g. @RT
file = file.drop_duplicates(subset="combined_text")

In [7]:
# print(file.shape)
# file.head(1)

### Process text and time colum

In [8]:
time_dict = {'Jan': "01",
            'Feb': "02",
            'Mar': "03",
            'Apr': "04",
            'May': "05",
            'Jun': "06",
            'Jul': "07",
            'Aug': "08",
            'Sep': "09", 
            'Oct': "10",
            'Nov': "11",
            'Dec': "12"}

In [9]:
time = []
for i in tqdm(range(len(file))):
    time_sep = file.iloc[i]["created_at"].split(" ")
    month = time_dict[time_sep[1]]
    day = time_sep[2]
    year = time_sep[-1]
    time.append(f"{month}-{day}-{year}")

100%|██████████| 174443/174443 [00:16<00:00, 10427.22it/s]


In [10]:
#copy df
file_new = file.copy()

#insert time column
file_new.insert(2, "time", time)

#drop useless column
file_new = file_new.drop(["created_at", "text", "full_text", "full_text.1", "full_text.2", "name", "screen_name", "id.1", "description", "country", "full_name", "place_type"], axis=1)

#rename column
file_new = file_new.rename(columns={"id":"tweet_id"})



In [11]:
# file_new.head(1)

,tweet_id,combined_text,time,favourites_count,followers_count,friends_count,statuses_count,location,rt_status
0,1223471115579711488,RT @FuschiaScribe: How come Baba Ramdev and a ...,02-01-2020,8420,94,518,6887,"Bengaluru, India",1


## Text data processing

In [12]:
states_dict = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}

In [13]:
states = [ 'AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA',
           'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME',
           'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM',
           'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
           'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY',
         'Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 
          'Connecticut', 'Washington DC', 'Delaware', 'Florida', 'Georgia', 'Hawaii', 
          'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 
          'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 
          'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina', 
          'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina', 
          'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 
          'Wisconsin', 'Wyoming',  'United States', 'USA', 'usa']
patt = '|'.join(states)
mask = file_new[['location']].apply(lambda x : x.str.contains(patt)).any(axis=1)
file_new = file_new[mask]

In [14]:
# print(file_new.shape)
# file_new.head(1)

In [15]:
state_list = []
for i in tqdm(range(len(file_new))):
    location = file_new.iloc[i]["location"]
    for j in states:
        if j in location:
            if j in states_dict.keys():
                state_list.append(states_dict[j])
            elif j in states_dict.values():
                state_list.append(j)
            else:
                state_list.append("")
            break

100%|██████████| 41707/41707 [00:03<00:00, 10553.23it/s]


In [16]:
file_new["state"] = state_list
# print(file_new.shape)
# file_new.head(1)

### Topic Modeling

In [17]:
# replaces spaces in keywords with connector matcher.
def replace_connector(keyword):
  connector = "[ -_]?"
  return connector.join(keyword.split())

#Filter tweets when keywords mention both covid and vaccine(not specify vaccine name)
covid_keywords = ["people", "new", "case", "rt", "virus", "19", "corona", "covid 19", "covid 2019", "covid", "19 ncov", "2019 ncov", "ncov", "corona virus", "pandemic", "wuhan virus", "chinese virus", "china virus", "sars cov 2", "hcov 19", "ncov 19"]
covid_query = r"\b(" + "|".join(map(replace_connector, covid_keywords)) + r")\b"
covid_reg = re.compile(covid_query, re.IGNORECASE)

In [18]:
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

In [19]:
tweet_tokenizer = TweetTokenizer()
tweet_tokens = []
# stops = set(stopwords.words('english'))
punctuation = """!"#$%&'()*+, -./:;<=>?“@[\]^_`{|}~’”—...'"""
lemmatizer = WordNetLemmatizer()
for sent in tqdm(list(file_new["combined_text"])):
    word_list = []
    for word in tweet_tokenizer.tokenize(sent):
        # prevent the case: lemmatize("Cases")==Cases
        word = word.lower()
        word = lemmatizer.lemmatize(word, get_wordnet_pos(word))
        if word not in STOP_WORDS and word not in punctuation and not covid_reg.search(word) and len(word) > 1: 
            word_list.append(word)
    tweet_tokens.append(word_list)

100%|██████████| 41707/41707 [01:58<00:00, 350.55it/s]


In [20]:
file_new["token_list"] = tweet_tokens

In [21]:
# print(file_new.shape)
# file_new.head(1)

In [24]:
file_new = file_new.drop(["location"], axis=1)
print(file_new.shape)
file_new.head(1)

(41707, 10)


,tweet_id,combined_text,time,favourites_count,followers_count,friends_count,statuses_count,rt_status,state,token_list
2,1223471140317909004,Veteran journo canned after making ‘racist’ jo...,02-01-2020,28128,4478,4826,42564,0,Tennessee,"[veteran, journo, racist, joke, https://t.co/v..."


In [25]:
file_new.to_csv("final_data/2020-02.csv", index=False)